In [3]:
import pickle
import pandas as pd
import numpy as np
from datetime import datetime

In [4]:
with open('pickles/parsed_big_df.pickle', 'rb') as data:
    df = pickle.load(data)

In [2]:
with open('models/best_rfc_fin.pickle', 'rb') as data:
    rfc_fin = pickle.load(data)
    
with open('models/best_rfc_inv.pickle', 'rb') as data:
    rfc_inv = pickle.load(data)

NameError: name 'pickle' is not defined

In [5]:
df.head()

,id,url,dateRFC,date,title,intro,text,text_parsed
0,2249,https://vc.ru/tribuna/2249-abrt3,"Tue, 28 Jan 2014 15:46:00 +0400",1390909560,Анализ стартапов от ABRT: Hudway vs. XPages Dy...,Возрождаем забытую практику разбора стартапов ...,Возрождаем забытую практику разбора стартапов ...,анализ стартап abrt hudway xpages dynamic возр...
1,2626,https://vc.ru/flood/2626-riders,"Mon, 27 Jan 2014 11:28:42 +0400",1390807722,RIDERS - глобальное комьюнити экстремалов,Игорь Дебатур и Анатолий Черняков рассказали п...,Игорь Дебатур и Анатолий Черняков рассказали п...,riders глобальный комьюнить экстремалов игорь ...
2,2657,https://vc.ru/flood/2657-evernote,"Tue, 07 Jan 2014 12:31:05 +0400",1389083465,CEO Evernote покаялся и обещает новый дизайн с...,"Фил Либин, основатель и CEO Evernote, признал ...","Фил Либин, основатель и CEO Evernote, признал ...",evernote покаяться обещать новый дизайн сервис...
3,2658,https://vc.ru/flood/2658-nvidia,"Wed, 08 Jan 2014 15:07:20 +0400",1389179240,Nvidia показали стриминг с ПК на телевизор,Nvidia продемонстрировали на пресс-мероприятии...,Nvidia продемонстрировали на пресс-мероприятии...,nvidia показать стриминг телевизор nvidia прод...
4,2660,https://vc.ru/tribuna/2660-startup-tv-series,"Wed, 08 Jan 2014 11:15:49 +0400",1389165349,"Четыре шоу о стартапах, которые вы пропустили",Если вас тоже порядком напрягает истерия вокру...,Если вас тоже порядком напрягает истерия вокру...,четыре стартап который пропустить если тоже по...


In [60]:
# Parameter election
ngram_range = (1, 1)
min_df = 10
max_df = 1.
max_features = 8000

tfidf = TfidfVectorizer(encoding='utf-8',
                        ngram_range=ngram_range,
                        stop_words=None,
                        lowercase=False,
                        max_df=max_df,
                        min_df=min_df,
                        max_features=max_features,
                        norm='l2',
                        sublinear_tf=True)

features = tfidf.fit_transform(df['text_parsed']).toarray()

# TF-IDF object
with open('pickles/tfidf.pickle', 'wb') as output:
    pickle.dump(tfidf, output)

In [1]:
fin_pred = rfc_fin.predict(features)

NameError: name 'rfc_fin' is not defined

In [63]:
inv_pred = rfc_inv.predict(features)

In [64]:
np.unique(fin_pred, return_counts=True)

(array([0, 1], dtype=int64), array([53091,  1731], dtype=int64))

In [65]:
np.unique(inv_pred, return_counts=True)

(array([0, 1], dtype=int64), array([54198,   624], dtype=int64))

In [66]:
tfidf

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=False, max_df=1.0, max_features=8000,
                min_df=10, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=True, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [80]:
fin = [0] * 72
inv = [0] * 72
total = [0] * 72
for i in range(len(df)):
    date = datetime.fromtimestamp(df.loc[i]['date'])
    j = (date.year - 2014) * 12 + date.month - 1 
    total[j] += 1
    if fin_pred[i] == 1:
        fin[j] += 1
    if inv_pred[i] == 1:
        inv[j] += 1        

In [85]:
data = {'Total' : total, 'Fintech' : fin, 'Investment' : inv} 

In [86]:
out = pd.DataFrame(data) 

In [87]:
out.head()

,Total,Fintech,Investment
0,192,2,0
1,216,4,0
2,254,2,1
3,318,9,2
4,295,8,8


In [88]:
out.to_csv('final.csv', sep=';')

In [5]:
tot_res = [0] * 72
fin_res = [0] * 72
inv_res = [0] * 72
words_tf_fin = ['финтех', 'криптовалюта', 'блокчейн', 'краудфандинг']
words_tf_fin_p = [('инвестиция','стартап'),('инвестиция', 'финтех'),('онлайн', 'банк'), ('финансовый', 'рынок')]
words_tf_inv = [('венчурный', 'капитал'), ('венчурный', 'финансирование')]
for i in range(len(df)):
    date = datetime.fromtimestamp(df.loc[i]['date'])
    j = (date.year - 2014) * 12 + date.month - 1 
    tot_res[j] += 1
    f = False
    for w in words_tf_fin:
        if df.loc[i]['text_parsed'].find(w) != -1:
            f = True
            fin_res[j] += 1
            break
    if f == False:    
        for w1, w2 in words_tf_fin_p:
            if df.loc[i]['text_parsed'].find(w1) != -1 and df.loc[i]['text_parsed'].find(w2) != -1:
                fin_res[j] += 1
                break
    for w1, w2 in words_tf_inv:
            if df.loc[i]['text_parsed'].find(w1) != -1 and df.loc[i]['text_parsed'].find(w2) != -1:
                inv_res[j] += 1
                break

In [6]:
data = {'Total' : tot_res, 'Fintech' : fin_res, 'Investment': inv_res} 

In [7]:
out = pd.DataFrame(data) 

In [8]:
out.to_csv('vc_key_words.csv', sep=';')